In [44]:
import time

import cv2
import numpy as np
import imutils

MIN_MATCH_COUNT = 10
start = time.time()
img1 = cv2.imread('../images/left1.png', 0)  # queryImage
img2 = cv2.imread('../images/right1.png', 0)  # trainImage
img1 = imutils.resize(img1, width=800)
img2 = imutils.resize(img2, width=800)

# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

FLANN_INDEX_KDTREE = 2
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# store all the good matches as per Lowe's ratio test.
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)

if len(good) > MIN_MATCH_COUNT:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist()

    h, w = img1.shape
    pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)

    img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

else:
    print("Not enough matches are found - %d/%d" % (len(good), MIN_MATCH_COUNT))
    matchesMask = None

draw_params = dict(matchColor=(0, 255, 0),  # draw matches in green color
                   singlePointColor=None,
                   matchesMask=matchesMask,  # draw only inliers
                   flags=2)

img3 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)
print("Operational Time=",(time.time()-start)*1000," ms") # Time in milli sec
cv2.imshow("image",img3)
#plt.imshow(img3)
if cv2.waitKey(0):
    cv2.destroyAllWindows()
    

Operational Time= 1010.141134262085  ms


In [9]:
# Python 4.3.0
# BF technique
from __future__ import print_function
import cv2
import numpy as np
import time
import imutils
start = time.time()
img1 = cv2.imread('../images/left1.png', 0)  # queryImage
img2 = cv2.imread('../images/right1.png', 0)  # trainImage
img1 = imutils.resize(img1, width=800)
img2 = imutils.resize(img2, width=800)
if img1 is None or img2 is None:
    print('Could not open or find the images!')
    exit(0)
#-- Step 1: Detect the keypoints using SURF Detector, compute the descriptors
minHessian = 400
detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
keypoints2, descriptors2 = detector.detectAndCompute(img2, None)
#-- Step 2: Matching descriptor vectors with a brute force matcher
# Since SURF is a floating-point descriptor NORM_L2 is used
matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_BRUTEFORCE)
matches = matcher.match(descriptors1, descriptors2)
#-- Draw matches
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches, img_matches)
#-- Show detected matches
print("Operational Time=",(time.time()-start)*1000," ms") # Time in milli sec
cv2.imshow('Matches', img_matches)
if cv2.waitKey(0):
    cv2.destroyAllWindows()

Operational Time= 890.1157379150391  ms


In [40]:
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.imshow("image",img2[:,:int((img2.shape[1])*0.45)])
if cv2.waitKey(0):
    cv2.destroyAllWindows()

In [37]:
int((img1.shape[1])*0.65)

2662

In [52]:
# FLANN Based
from __future__ import print_function
import cv2
import numpy as np
import imutils
import time
start = time.time()
img1 = cv2.imread('../images/left1.png', 0)  # queryImage
img2 = cv2.imread('../images/right1.png', 0)  # trainImage
img1 = imutils.resize(img1, width=800)
img2 = imutils.resize(img2, width=800)

if img1 is None or img2 is None:
    print('Could not open or find the images!')
    exit(0)

#-- Step 1: Detect the keypoints using SURF Detector, compute the descriptors
minHessian = 400
detector = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
keypoints1, descriptors1 = detector.detectAndCompute(img1, None)
keypoints2, descriptors2 = detector.detectAndCompute(img2, None)

#-- Step 2: Matching descriptor vectors with a FLANN based matcher
# Since SURF is a floating-point descriptor NORM_L2 is used
matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
knn_matches = matcher.knnMatch(descriptors1, descriptors2, 2)

#-- Filter matches using the Lowe's ratio test
ratio_thresh = 0.7
good_matches = []
for m,n in knn_matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)

#-- Draw matches
img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
cv2.drawMatches(img1, keypoints1, img2, keypoints2, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

print("Operational Time=",(time.time()-start)*1000," ms") # Time in milli sec
#-- Show detected matches
cv2.imshow('Good Matches', img_matches)

if cv2.waitKey(0):
    cv2.destroyAllWindows()

Operational Time= 826.1146545410156  ms
